In [1]:
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'
os.environ["WANDB_API_KEY"] = "6f810b088fcc6b9eaaa56c1e52cfd37836606240"

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q tensorflow==2.6.2
!pip install -q tensorflow-recommenders==0.6.0
!pip install -q scann==1.2.3
!pip install -q tensorflow-datasets
!pip install -q tensorflow-addons
!pip install -q tensorflow-hub
!pip install -q keras-tuner
!pip install -q statsmodels
!pip install -q optuna
!pip install -q wandb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1
!pip install rhinoMorph

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [5]:
import sys
import shutil
from glob import glob
from IPython.display import Image, display
import gc
import wandb

from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random as rnd
import psutil
from optuna import Trial, create_study
from optuna.samplers import TPESampler

import numpy as np
from numpy import array, nan, random as np_rnd, where as which
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset

from sklearn.model_selection import train_test_split as tts, GridSearchCV as GridTuner, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, KBinsDiscretizer
from sklearn import metrics
# from optuna import Trial, create_study

# ===== tensorflow =====
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow_recommenders.tasks import base
from keras.utils.layer_utils import count_params

import keras_tuner as kt
from keras_tuner import HyperModel
import tensorflow_hub as tf_hub

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import scann

import rhinoMorph
from kobert_tokenizer import KoBERTTokenizer
import sentencepiece
from transformers import BertModel
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)

# GPU check
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

Found GPU at: /device:GPU:0


# Utility Funtions

In [6]:
# ===== utility functions =====
# label encoding for categorical column with excepting na value
def seed_everything(seed=42):
    # python-self
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random module
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    tf.random.set_seed(seed)
    # RAPIDS random
    try:
        cp.random.seed(seed)
    except:
        pass
def easyIO(x=None, path=None, op="r", keras_inspection=False):
    tmp = None
    if op == "r":
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        return tmp
    elif op == "w":
        print(x)
        tmp = x
        if keras_inspection:
            tmp = {}
            if type(x) is dict:
                for k in x.keys():
                    if "MLP" in k:
                        tmp[k] = {}
                        for model_comps in x[k].keys():
                            if model_comps != "model":
                                tmp[k][model_comps] = copy.deepcopy(x[k][model_comps])
                        print(F"INFO : {k} model is removed (keras)")
                    else:
                        tmp[k] = x[k]
        if input("Write [y / n]: ") == "y":
            with open(path, "wb") as f:
                pickle.dump(tmp, f)
            print("operation success")
        else:
            print("operation fail")
    else:
        print("Unknown operation type")
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]
def orderElems(for_order, using_ref):
    return [i for i in using_ref if i in for_order]
# concatenate by row
def cbr(df1, df2):
    if type(df1) == series:
        tmp_concat = series(pd.concat([dataframe(df1), dataframe(df2)], axis=0, ignore_index=True).iloc[:,0])
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == dataframe:
        tmp_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == np.ndarray:
        tmp_concat = np.concatenate([df1, df2], axis=0)
    else:
        print("Unknown Type: return 1st argument")
        tmp_concat = df1
    return tmp_concat
def change_width(ax, new_value):
    for patch in ax.patches :
        current_width = patch.get_width()
        adj_value = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + adj_value * .5)
def week_of_month(date):
    month = date.month
    week = 0
    while date.month == month:
        week += 1
        date -= timedelta(days=7)
    return week
def getSeason(date):
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else:
        return "Winter"
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
def softmax(x):
    max = np.max(x, axis=1, keepdims=True)  # returns max of each row and keeps same dims
    e_x = np.exp(x - max)  # subtracts each row with its max value
    sum = np.sum(e_x, axis=1, keepdims=True)  # returns sum of each row and keeps same dims
    f_x = e_x / sum
    return f_x
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def dispPerformance(result_dic):
    perf_table = dataframe()
    index_names = []
    for k, v in result_dic.items():
        index_names.append(k)
        perf_table = pd.concat([perf_table, series(v["performance"]).to_frame().T], ignore_index=True, axis=0)
    perf_table.index = index_names
    perf_table.sort_values(perf_table.columns[0], inplace=True)
    print(perf_table)
    return perf_table
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power)
def xgb_custom_lossfunction(alpha = 1):
    def support_under_mse(label, pred):
        # grad : 1차 미분
        # hess : 2차 미분
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return support_under_mse
def pd_flatten(df):
    df = df.unstack()
    df.index = [str(i) + "_" + str(j) for i, j in df.index]
    return df
def tf_loss_rmse(y_true, y_pred, sample_weight=False):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred) ** 2)) if not sample_weight else tf.sqrt(tf.reduce_mean(((y_true - y_pred) ** 2) * sample_weight))
def text_extractor(string, lang="eng", spacing=True):
    # # 괄호를 포함한 괄호 안 문자 제거 정규식
    # re.sub(r'\([^)]*\)', '', remove_text)
    # # <>를 포함한 <> 안 문자 제거 정규식
    # re.sub(r'\<[^)]*\>', '', remove_text)
    if lang == "eng":
        text_finder = re.compile('[^ A-Za-z]') if spacing else re.compile('[^A-Za-z]')
    elif lang == "kor":
        text_finder = re.compile('[^ ㄱ-ㅣ가-힣+]') if spacing else re.compile('[^ㄱ-ㅣ가-힣+]')
    # default : kor + eng
    else:
        text_finder = re.compile('[^ A-Za-zㄱ-ㅣ가-힣+]') if spacing else re.compile('[^A-Za-zㄱ-ㅣ가-힣+]')
    return text_finder.sub('', string)
class MyLabelEncoder:
    def __init__(self, preset={}):
        # dic_cat format -> {"col_name": {"value": replace}}
        self.dic_cat = preset
    def fit_transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            # if key is not in dic, update dic
            if i not in self.dic_cat.keys():
                tmp_dic = dict.fromkeys(sorted(set(tmp_x[i]).difference([nan])))
                label_cnt = 0
                for j in tmp_dic.keys():
                    tmp_dic[j] = label_cnt
                    label_cnt += 1
                self.dic_cat[i] = tmp_dic
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def transform(self, data_x):
        tmp_x = copy.deepcopy(data_x)
        for i in self.dic_cat.keys():
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def clear(self):
        self.dic_cat = {}
class MyOneHotEncoder:
    def __init__(self, label_preset={}):
        self.dic_cat = {}
        self.label_preset = label_preset
    def fit_transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if i not in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                self.dic_cat[i] = OneHotEncoder(sparse=False, handle_unknown="ignore")
                conv = self.dic_cat[i].fit_transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j:
                                col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def transform(self, data_x):
        tmp_x = dataframe()
        for i in data_x:
            if not i in list(self.dic_cat.keys()):
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                conv = self.dic_cat[i].transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j: col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def clear(self):
        self.dic_cat = {}
        self.label_preset = {}
class MyKNNImputer:
    def __init__(self, k=5):
        self.imputer = KNNImputer(n_neighbors=k)
        self.dic_cat = {}
    def fit_transform(self, x, cat_vars=None):
        if cat_vars is None:
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(cat_vars)
            for i in cat_vars:
                self.dic_cat[i] = diff(list(sorted(set(x[i]))), [nan])
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in cat_vars:
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def transform(self, x):
        if len(self.dic_cat.keys()) == 0:
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(self.dic_cat.keys())
            for i in self.dic_cat.keys():
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in self.dic_cat.keys():
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def clear(self):
        self.imputer = None
        self.dic_cat = {}

def memory_usage(message='debug'):
    # current process RAM usage
    p = psutil.Process()
    rss = p.memory_info().rss / 2 ** 20 # Bytes to MB
    print(f"[{message}] memory usage: {rss: 10.3f} MB")
    return rss

# Preprocessing & Feature Engineering

In [7]:
# 데이터 로딩
def read_data(filename, encoding='utf8'):
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('|') for line in f.read().splitlines()]
  return data

def write_data(data, filename, encoding='utf8'):
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

folder_path = '/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/'

# header info : AI_id|digit_1|digit_2|digit_3|text_obj|text_mthd|text_deal
full_x = read_data('/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/1. 실습용자료.txt', encoding='cp949')[1:]
test_x = read_data('/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/2. 모델개발용자료.txt', encoding='cp949')[1:]

In [8]:
full_x = dataframe(full_x, columns=["AI_id", "digit_1", "digit_2", "digit_3", "text_obj", "text_mthd", "text_deal"])
test_x = dataframe(test_x, columns=["AI_id", "digit_1", "digit_2", "digit_3", "text_obj", "text_mthd", "text_deal"])

In [9]:
full_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   AI_id      1000000 non-null  object
 1   digit_1    1000000 non-null  object
 2   digit_2    1000000 non-null  object
 3   digit_3    1000000 non-null  object
 4   text_obj   1000000 non-null  object
 5   text_mthd  1000000 non-null  object
 6   text_deal  1000000 non-null  object
dtypes: object(7)
memory usage: 53.4+ MB


In [10]:
full_x.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육


In [11]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   AI_id      100000 non-null  object
 1   digit_1    100000 non-null  object
 2   digit_2    100000 non-null  object
 3   digit_3    100000 non-null  object
 4   text_obj   100000 non-null  object
 5   text_mthd  100000 non-null  object
 6   text_deal  100000 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB


In [12]:
test_x.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,,,,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,,,,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,,,,절에서,신도을 대상으로,불교단체운영
3,id_000004,,,,영업장에서,고객요구로,자동차튜닝
4,id_000005,,,,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"


In [13]:
# full_x["sentence"] = full_x["text_obj"] + " " + full_x["text_mthd"] + " " + full_x["text_deal"]
# test_x["sentence"] = test_x["text_obj"] + " " + test_x["text_mthd"] + " " + test_x["text_deal"]

In [14]:
# full_x["sentence"] = full_x["sentence"].apply(lambda x: text_extractor(x, "both", True))
# test_x["sentence"] = test_x["sentence"].apply(lambda x: text_extractor(x, "both", True))

In [15]:
full_x.drop(["AI_id", "text_obj", "text_mthd", "text_deal"], axis=1, inplace=True)
test_x.drop(["AI_id", "text_obj", "text_mthd", "text_deal"], axis=1, inplace=True)

In [16]:
# rn = rhinoMorph.startRhino()

# if os.path.exists(folder_path + "train_prep.txt"):
#     os.remove(folder_path + "train_prep.txt")
# with open(folder_path + "train_prep.txt", 'a', encoding="cp949") as f:
#     for i in full_x["sentence"]:
#         f.write(" ".join([j.lower() for j in rhinoMorph.onlyMorph_list(rn, i) if len(j) > 1]) + "\n")

# if os.path.exists(folder_path + "test_prep.txt"):
#     os.remove(folder_path + "test_prep.txt")
# with open(folder_path + "test_prep.txt", 'a', encoding="cp949") as f:
#     for i in test_x["sentence"]:
#         f.write(" ".join([j.lower() for j in rhinoMorph.onlyMorph_list(rn, i) if len(j) > 1]) + "\n")

In [17]:
with open(folder_path + "train_prep.txt", 'r', encoding="cp949") as f:
    full_x["sentence"] = [i.strip() for i in f.readlines()]
with open(folder_path + "test_prep.txt", 'r', encoding="cp949") as f:
    test_x["sentence"] = [i.strip() for i in f.readlines()]

In [18]:
full_x.head()

,digit_1,digit_2,digit_3,sentence
0,S,95,952,카센터 에서 자동차 부분 정비 타이어 오일 교환
1,G,47,472,상점 에서 일반인 대상 으로 채소 과일 판매
2,G,46,467,절단 사업체 에도 공업 용고무를 가지 합성고무 도매
3,G,47,475,영업점 에서 일반 소비자 에게 열쇠 잠금장치
4,Q,87,872,어린이집 보호자 위탁 취학 아동 보육


In [19]:
test_x.head()

,digit_1,digit_2,digit_3,sentence
0,,,,치킨 전문점 에서 고객 주문 치킨 판매
1,,,,산업 공구 다른 소매업자 에게 철물 수공
2,,,,에서 신도 대상 으로 불교 단체 운영
3,,,,영업장 에서 고객 요구 자동차 튜닝
4,,,,실내 포장마차 에서 접객 시설 갖추 소주 맥주 제공


In [20]:
target_encoder = MyLabelEncoder()
full_y = full_x[["digit_1", "digit_2", "digit_3"]].copy()
full_y = target_encoder.fit_transform(full_y, ["digit_1", "digit_2", "digit_3"]).astype("int32")

In [21]:
for i in target_encoder.dic_cat.keys():
    print(len(target_encoder.dic_cat[i].keys()))

19
74
225


In [22]:
full_x.drop(["digit_1", "digit_2", "digit_3"], axis=1, inplace=True)
test_x.drop(["digit_1", "digit_2", "digit_3"], axis=1, inplace=True)

In [23]:
full_x.head()

,sentence
0,카센터 에서 자동차 부분 정비 타이어 오일 교환
1,상점 에서 일반인 대상 으로 채소 과일 판매
2,절단 사업체 에도 공업 용고무를 가지 합성고무 도매
3,영업점 에서 일반 소비자 에게 열쇠 잠금장치
4,어린이집 보호자 위탁 취학 아동 보육


In [24]:
full_y.head()

,digit_1,digit_2,digit_3
0,18,72,221
1,6,37,120
2,6,36,117
3,6,37,123
4,16,68,212


In [25]:
test_x.head()

,sentence
0,치킨 전문점 에서 고객 주문 치킨 판매
1,산업 공구 다른 소매업자 에게 철물 수공
2,에서 신도 대상 으로 불교 단체 운영
3,영업장 에서 고객 요구 자동차 튜닝
4,실내 포장마차 에서 접객 시설 갖추 소주 맥주 제공


In [26]:
bool_mask = []
for i in full_y:
    # print(full_y[i].value_counts())
    # print("\n")
    tmp_vc = full_y[i].value_counts()
    low_freq = tmp_vc.index[tmp_vc <= 3]
    if len(low_freq) > 0:
        bool_mask.append(full_y[i].apply(lambda x: False if x in low_freq else True))
bool_mask = (bool_mask[0] & bool_mask[1])

**Drop minor class samples (num_classes <= 3)**

In [27]:
full_y = full_y[bool_mask]
full_x = full_x[bool_mask]

In [28]:
def create_model_vectorization(input_shape, max_vocab, adapt_corpus, output_mode="int", output_sequence_length=None):
    vectorizer = layers.TextVectorization(
            max_tokens=max_vocab,
            output_mode=output_mode,
            output_sequence_length=output_sequence_length,
            standardize=None,
            name="tv"
    )
    vectorizer.adapt(adapt_corpus)
    model = Sequential([
        layers.InputLayer(input_shape=input_shape, dtype=tf.string),
        vectorizer
    ])
    return model

In [29]:
# # with tf.device('/CPU:0'):
# #     max_vocab = 8192
# #     doc_vectorize = create_model_vectorization(1, max_vocab, full_x["sentence"].to_numpy(), output_mode="tf_idf", output_sequence_length=None)
# #     print(doc_vectorize.get_layer("tv").vocabulary_size())
# #     doc_vectorize.compile()
# #     doc_vectorize.save(os.path.join(folder_path, "models/doc_vectorizer_vocab8192"))
# doc_vectorize = load_model(os.path.join(folder_path, "models/doc_vectorizer_vocab4096"))
# print(doc_vectorize.get_layer("tv").vocabulary_size())

In [30]:
# with tf.device('/CPU:0'):
#     full_doc_embed = doc_vectorize(full_x["sentence"].to_numpy()).numpy()
#     test_doc_embed = doc_vectorize(test_x["sentence"].to_numpy()).numpy()

In [31]:
# full_doc_embed = pad_sequences(full_doc_embed, padding='pre')
# test_doc_embed = pad_sequences(test_doc_embed, padding='pre')

In [32]:
#### learning parameter setting
epochs = 50
patient_epochs = max(int(epochs * 0.2), 10)
patient_lr = 2
batch_size = 256
eta = 1e-3
weight_decay = 1e-4
checkpoint_filepath = './tmp_checkpoint/fold_checkpoint'
optimizer_name = "AdamW"

result_prob = dict.fromkeys(["digit_1", "digit_2", "digit_3"])
result_prob["digit_1"] = np.zeros(shape=(test_x.shape[0], len(target_encoder.dic_cat["digit_1"].keys())))
result_prob["digit_2"] = np.zeros(shape=(test_x.shape[0], len(target_encoder.dic_cat["digit_2"].keys())))
result_prob["digit_3"] = np.zeros(shape=(test_x.shape[0], len(target_encoder.dic_cat["digit_3"].keys())))

In [33]:
def create_dataset(x, y, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(int(batch_size * 2), reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2)
    return dataset

# NN models

In [34]:
tfidf_size = 4096
dropoutRate = 0.5

In [35]:
# # base model
# def create_model():
#     # model_input = layers.Input(shape=doc_vectorize.get_layer("tv").vocabulary_size(), dtype=tf.float32)
#     # x = layers.Dropout(0.5)(model_input)
#     # x = layers.Dense(512, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(256, activation="relu")(x)
#     # input_dense1 = layers.Dropout(0.5)(x)

#     # model_input = layers.Input(shape=doc_vectorize.get_layer("tv").vocabulary_size(), dtype=tf.float32)
#     # x = layers.Dropout(0.5)(model_input)
#     # x = layers.Dense(1024, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(512, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(256, activation="relu")(x)
#     # input_dense1 = layers.Dropout(0.5)(x)

#     model_input = layers.Input(shape=tfidf_size, dtype=tf.float32)
#     x = tf.keras.layers.GaussianNoise(0.5)(model_input)
#     x = tf.keras.layers.ReLU()(x)
#     x = layers.Dense(2048, activation="relu", activity_regularizer="l2")(x)
#     x = layers.Dropout(dropoutRate)(x)
#     x = layers.Dense(1024, activation="relu")(x)
#     x = layers.Dropout(dropoutRate)(x)
#     x = layers.Dense(512, activation="relu")(x)
#     x = layers.Dropout(dropoutRate)(x)
#     x = layers.Dense(256, activation="relu")(x)
#     input_dense1 = layers.Dropout(dropoutRate)(x)

#     # model_input = layers.Input(shape=doc_vectorize.get_layer("tv").vocabulary_size(), dtype=tf.float32)
#     # x = layers.Dropout(0.5)(model_input)
#     # x = layers.Dense(4096, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(2048, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(1024, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(512, activation="relu")(x)
#     # x = layers.Dropout(0.5)(x)
#     # x = layers.Dense(256, activation="relu")(x)
#     # input_dense1 = layers.Dropout(0.5)(x)

#     x = tfa.layers.WeightNormalization(
#         layers.Dense(128, activation="relu")
#     )(input_dense1)
#     x = layers.Dropout(dropoutRate)(x)
#     input_dense2 = layers.Concatenate()([input_dense1, x])

#     x = tfa.layers.WeightNormalization(
#         layers.Dense(128, activation="relu")
#     )(input_dense2)
#     x = layers.Dropout(dropoutRate)(x)
#     input_dense3 = layers.Concatenate()([input_dense2, x])

#     x = tfa.layers.WeightNormalization(
#         layers.Dense(128, activation="relu")
#     )(input_dense3)
#     x = layers.Dropout(dropoutRate)(x)

#     x = tfa.layers.WeightNormalization(
#         layers.Dense(64, activation="swish")
#     )(x)

#     classifier_digit1 = layers.Dense(len(target_encoder.dic_cat["digit_1"].keys()), activation="softmax")(x)
#     classifier_digit2 = layers.Dense(len(target_encoder.dic_cat["digit_2"].keys()), activation="softmax")(x)
#     classifier_digit3 = layers.Dense(len(target_encoder.dic_cat["digit_3"].keys()), activation="softmax")(x)
#     return Model(model_input, [classifier_digit1, classifier_digit2, classifier_digit3])

In [36]:
tfidf_size = 4096
dropoutRate = 0.5

In [37]:
# last dense layer is seperated
def create_model():
    model_input = layers.Input(shape=tfidf_size, dtype=tf.float32)
    x = tf.keras.layers.GaussianNoise(3)(model_input)
    x = tf.keras.layers.ReLU()(x)
    # x = layers.Dense(2048, activation="relu", activity_regularizer="l2")(x)
    # x = layers.Dropout(dropoutRate)(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)
    x = layers.Dropout(dropoutRate)(x)
    # x = layers.Dense(512, activation="relu")(x)
    # x = layers.Dropout(dropoutRate)(x)
    x = layers.Dense(256, activation="relu")(x)
    input_dense1 = layers.Dropout(dropoutRate)(x)

    x = tfa.layers.WeightNormalization(
        layers.Dense(128, activation="relu")
    )(input_dense1)
    x = layers.Dropout(dropoutRate)(x)

    input_dense2 = layers.Concatenate()([input_dense1, x])
    x = tfa.layers.WeightNormalization(
        layers.Dense(128, activation="relu")
    )(input_dense2)
    x = layers.Dropout(dropoutRate)(x)

    input_dense3 = layers.Concatenate()([input_dense2, x])
    x = tfa.layers.WeightNormalization(
        layers.Dense(128, activation="relu")
    )(input_dense3)
    x = layers.Dropout(dropoutRate)(x)
    dense_digit1 = tfa.layers.WeightNormalization(
        layers.Dense(64, activation="swish")
    )(x)

    x = tfa.layers.WeightNormalization(
        layers.Dense(256, activation="relu")
    )(input_dense3)
    x = layers.Dropout(dropoutRate)(x)
    dense_digit2 = tfa.layers.WeightNormalization(
        layers.Dense(128, activation="swish")
    )(x)

    x = tfa.layers.WeightNormalization(
        layers.Dense(512, activation="relu")
    )(input_dense3)
    x = layers.Dropout(dropoutRate)(x)
    dense_digit3 = tfa.layers.WeightNormalization(
        layers.Dense(256, activation="swish")
    )(x)

    classifier_digit1 = layers.Dense(len(target_encoder.dic_cat["digit_1"].keys()), activation="softmax")(dense_digit1)
    classifier_digit2 = layers.Dense(len(target_encoder.dic_cat["digit_2"].keys()), activation="softmax")(dense_digit2)
    classifier_digit3 = layers.Dense(len(target_encoder.dic_cat["digit_3"].keys()), activation="softmax")(dense_digit3)
    return Model(model_input, [classifier_digit1, classifier_digit2, classifier_digit3])

In [38]:
tfidf_size = 4096
dropoutRate = 0.5

In [39]:
# # all learning dense layers are seperated
# def create_model():
#     model_input = layers.Input(shape=tfidf_size, dtype=tf.float32)
#     x = tf.keras.layers.GaussianNoise(0.5)(model_input)
#     x = tf.keras.layers.ReLU()(x)
#     x = layers.Dense(2048, activation="relu", activity_regularizer="l2")(x)
#     x = layers.Dropout(dropoutRate)(x)
#     x = layers.Dense(1024, activation="relu")(x)
#     x = layers.Dropout(dropoutRate)(x)
#     x = layers.Dense(512)(x)
#     input_prelu1 = tf.keras.layers.PReLU()(x)
#     input_dense1 = layers.Dropout(dropoutRate)(input_prelu1)
#     input_prelu2 = tf.keras.layers.PReLU()(x)
#     input_dense2 = layers.Dropout(dropoutRate)(input_prelu2)
#     input_prelu3 = tf.keras.layers.PReLU()(x)
#     input_dense3 = layers.Dropout(dropoutRate)(input_prelu3)

#     tmp = []
#     for i in range(3):
#         x = tfa.layers.WeightNormalization(
#             layers.Dense(int(512 * (2+i)/4), activation="relu")
#         )(input_dense1)
#         x = layers.Dropout(dropoutRate)(x)
#         input_dense2 = layers.Concatenate()([input_dense1, x])

#         x = tfa.layers.WeightNormalization(
#             layers.Dense(int(512 * (2+i)/4), activation="relu")
#         )(input_dense2)
#         x = layers.Dropout(dropoutRate)(x)
#         input_dense3 = layers.Concatenate()([input_dense2, x])

#         x = tfa.layers.WeightNormalization(
#             layers.Dense(int(512 * (2+i)/4), activation="relu")
#         )(input_dense3)
#         tmp.append(layers.Dropout(dropoutRate)(x))
  
#     last_digit1 = tfa.layers.WeightNormalization(
#         layers.Dense(64, activation="swish")
#     )(layers.Concatenate()([tmp[0]]))
#     classifier_digit1 = layers.Dense(len(target_encoder.dic_cat["digit_1"].keys()), activation="softmax")(last_digit1)

#     last_digit1_dropout = layers.Dropout(dropoutRate)(last_digit1)
#     last_digit2 = tfa.layers.WeightNormalization(
#         layers.Dense(128, activation="swish")
#     )(layers.Concatenate()([last_digit1_dropout, tmp[1]]))
#     classifier_digit2 = layers.Dense(len(target_encoder.dic_cat["digit_2"].keys()), activation="softmax")(last_digit2)

#     last_digit2_dropout = layers.Dropout(dropoutRate)(last_digit2)
#     last_digit3 = tfa.layers.WeightNormalization(
#         layers.Dense(256, activation="swish")
#     )(layers.Concatenate()([last_digit1_dropout, last_digit2_dropout, tmp[2]]))
#     classifier_digit3 = layers.Dense(len(target_encoder.dic_cat["digit_3"].keys()), activation="softmax")(last_digit3)
#     return Model(model_input, [classifier_digit1, classifier_digit2, classifier_digit3])

In [40]:
# create_model().summary()

In [41]:
tmp_model = create_model()
model_size = (count_params(tmp_model.trainable_weights), count_params(tmp_model.non_trainable_weights))
createFolder(folder_path + "tmp_plot_model/")
model_img_path = folder_path + "tmp_plot_model/model_architecture.png"
plot_model(tmp_model, to_file=model_img_path,
           show_shapes=True, show_dtype=True, expand_nested=True, dpi=256)
del tmp_model

# Define Tasks

In [42]:
class task_classifier(tf.keras.layers.Layer, base.Task):
    def __init__(
        self,
        loss,
        metrics=None,
        prediction_metrics=None,
        label_metrics=None,
        loss_metrics=None,
        name=None):
        super().__init__(name=name)

        self._loss = (loss if loss is not None else tf.keras.losses.BinaryCrossentropy())
        self._ranking_metrics = metrics or []
        self._prediction_metrics = prediction_metrics or []
        self._label_metrics = label_metrics or []
        self._loss_metrics = loss_metrics or []

    def call(
        self,
        labels,
        predictions,
        sample_weight=None,
        training=False,
        compute_metrics=True):
        loss = self._loss(y_true=labels, y_pred=predictions, sample_weight=sample_weight)
        if not compute_metrics:
            return loss
        update_ops = []

        for metric in self._ranking_metrics:
            update_ops.append(metric.update_state(
                y_true=labels, y_pred=predictions, sample_weight=sample_weight))

        for metric in self._prediction_metrics:
            update_ops.append(
                metric.update_state(predictions, sample_weight=sample_weight))

        for metric in self._label_metrics:
            update_ops.append(
                metric.update_state(labels, sample_weight=sample_weight))

        for metric in self._loss_metrics:
            update_ops.append(
                metric.update_state(loss, sample_weight=sample_weight))

        # Custom metrics may not return update ops, unlike built-in
        # Keras metrics.
        update_ops = [x for x in update_ops if x is not None]

        with tf.control_dependencies(update_ops):
            return tf.identity(loss)

In [43]:
class task_weighted_metric(tf.keras.layers.Layer, base.Task):
    def __init__(
        self,
        loss=None,
        sample_weight=(0.1, 0.2, 0.7),
        metrics=None,
        prediction_metrics=None,
        label_metrics=None,
        loss_metrics=None,
        name=None):

        super().__init__(name=name)
        self._loss = (loss if loss is not None else tf.keras.losses.BinaryCrossentropy())
        self._ranking_metrics = metrics
        self._sample_weight = sample_weight

    def call(
        self,
        raw_scores,
        training=False,
        compute_metrics=True):

        # loss = self._loss(y_true=labels, y_pred=predictions, sample_weight=sample_weight)
        loss = 0.0
        if not compute_metrics:
            return loss
        update_ops = []
        for metric in self._ranking_metrics:
            if metric.name == "weighted_logloss":
                update_ops.append(metric.update_state(values=raw_scores[0], sample_weight=self._sample_weight))
            elif metric.name == "weighted_acc":
                update_ops.append(metric.update_state(values=raw_scores[1], sample_weight=self._sample_weight))
            elif metric.name == "weighted_f1score":
                update_ops.append(metric.update_state(values=raw_scores[2], sample_weight=self._sample_weight))

        # Custom metrics may not return update ops, unlike built-in
        # Keras metrics.
        update_ops = [x for x in update_ops if x is not None]
        with tf.control_dependencies(update_ops):
            return tf.identity(loss)

In [44]:
class Model_Mutlitask(tfrs.models.Model):
    def __init__(self, model_digit_base, loss_weight=(1.0, 1.0, 1.0)):
        # We take the loss weights in the constructor: this allows us to instantiate
        # several model objects with different loss weights.
        super().__init__()
        self.model_digit_base = model_digit_base
        # self.model_digit1 = create_classifier(len(target_encoder.dic_cat["digit_1"].keys()))
        # self.model_digit2 = create_classifier(len(target_encoder.dic_cat["digit_2"].keys()))
        # self.model_digit3 = create_classifier(len(target_encoder.dic_cat["digit_3"].keys()))
        self.task_digit1 = task_classifier(
            loss=tf.keras.losses.CategoricalCrossentropy(name="digit1_loss"),
            metrics=[tf.keras.metrics.CategoricalCrossentropy(name="digit1_logloss"),
                     tf.keras.metrics.CategoricalAccuracy(name="digit1_acc"),
                     tfa.metrics.F1Score(num_classes=len(target_encoder.dic_cat["digit_1"].keys()), average="macro", name="digit1_f1score")]
        )
        self.task_digit2 = task_classifier(
            loss=tf.keras.losses.CategoricalCrossentropy(name="digit2_loss"),
            metrics=[tf.keras.metrics.CategoricalCrossentropy(name="digit2_logloss"),
                     tf.keras.metrics.CategoricalAccuracy(name="digit2_acc"),
                     tfa.metrics.F1Score(num_classes=len(target_encoder.dic_cat["digit_2"].keys()), average="macro", name="digit2_f1score")]
        )
        self.task_digit3 = task_classifier(
            loss=tf.keras.losses.CategoricalCrossentropy(name="digit3_loss"),
            metrics=[tf.keras.metrics.CategoricalCrossentropy(name="digit3_logloss"),
                     tf.keras.metrics.CategoricalAccuracy(name="digit3_acc"),
                     tfa.metrics.F1Score(num_classes=len(target_encoder.dic_cat["digit_3"].keys()), average="macro", name="digit3_f1score")]
        )


        self.weighted_metrics = task_weighted_metric(
            metrics=[tf.keras.metrics.Mean(name="weighted_logloss"),
                     tf.keras.metrics.Mean(name="weighted_acc"),
                     tf.keras.metrics.Mean(name="weighted_f1score")]
        )
        self.loss_weight = loss_weight
    def compute_loss(self, features, training=False):
        softmax_digit1, softmax_digit2, softmax_digit3 = self.model_digit_base(features[0])
        loss_digit1 = self.task_digit1(predictions=softmax_digit1, labels=features[1][0], sample_weight=None)
        loss_digit2 = self.task_digit2(predictions=softmax_digit2, labels=features[1][1], sample_weight=None)
        loss_digit3 = self.task_digit3(predictions=softmax_digit3, labels=features[1][2], sample_weight=None)
        raw_scores = [
            (
                self.task_digit1._ranking_metrics[0].result(),
                self.task_digit2._ranking_metrics[0].result(),
                self.task_digit3._ranking_metrics[0].result()
            ),
            (
                self.task_digit1._ranking_metrics[1].result(),
                self.task_digit2._ranking_metrics[1].result(),
                self.task_digit3._ranking_metrics[1].result()
            ),
            (
                self.task_digit1._ranking_metrics[2].result(),
                self.task_digit2._ranking_metrics[2].result(),
                self.task_digit3._ranking_metrics[2].result()
            )
        ]
        self.weighted_metrics(raw_scores)

        return (loss_digit1*self.loss_weight[0] + loss_digit2*self.loss_weight[1] + loss_digit3*self.loss_weight[2])

# Fold Training

In [45]:
# optuna function
# loss_weight = ()
loss_weight = [0.4, 0.57, 1.32]

def optuna_objective_function(trial: Trial, train_ds, val_ds):
    seed_everything(8787)
    tuning_params = {
        "loss_weight1": trial.suggest_float("loss_weight1", 0.1, 1.8, step=0.01),
        "loss_weight2": trial.suggest_float("loss_weight2", 0.2, 1.9, step=0.01),
        "loss_weight3": trial.suggest_float("loss_weight3", 0.3, 2.0, step=0.01),
    }
    loss_weight = (tuning_params["loss_weight1"], tuning_params["loss_weight2"], tuning_params["loss_weight3"])

    cb_modelsave = tf_callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_weighted_acc', mode='max',
                                                save_weights_only=True, save_best_only=True)
    model = Model_Mutlitask(create_model(), loss_weight=loss_weight)
    model.compile(
        optimizer=tfa.optimizers.AdamW(learning_rate=1e-5, weight_decay=weight_decay)
    )
    history = model.fit(train_ds, validation_data=val_ds, epochs=10, verbose=0, callbacks=[cb_modelsave, TqdmCallback(verbose=0)])
    model.load_weights(checkpoint_filepath)

    eval_metric = model.evaluate(val_ds, return_dict=True, verbose=0)
    opt_score = eval_metric["weighted_acc"]
    return opt_score

In [46]:
model_name = "NN_tfidf_multi_task_learning"
wandb_group_name = "vocab4096_dropout0.5_weightdecay1e-4_allSeperated_try1"

In [47]:
if os.path.exists(os.path.join(folder_path, "models/" + model_name)):
    shutil.rmtree(os.path.join(folder_path, "models/" + model_name))
    createFolder(os.path.join(folder_path, "models/" + model_name))

In [48]:
def do_fold_training(fold, train_idx, val_idx):
    tmp_time = time()
    print("\n===== Fold ", fold, "=====\n")

    with tf.device('/CPU:0'):
        doc_vectorize = create_model_vectorization(1, tfidf_size, full_x["sentence"].iloc[train_idx].to_numpy(), output_mode="tf_idf", output_sequence_length=None)
        doc_vectorize.compile()
        doc_vectorize.save(os.path.join(folder_path, "models/" + model_name + "doc_vectorizer_vocab" + str(tfidf_size) + "_" + str(fold)))
        doc_vectorize = load_model(os.path.join(folder_path, "models/" + model_name + "doc_vectorizer_vocab" + str(tfidf_size) + "_" + str(fold)))
        tmp_doc_embed = pad_sequences(doc_vectorize(full_x["sentence"].to_numpy()[train_idx]).numpy(), padding='pre').astype(dtype="float32")
        
    tmp_target = (
        tf.keras.utils.to_categorical(full_y["digit_1"].iloc[train_idx].values, num_classes=len(target_encoder.dic_cat["digit_1"].keys())),
        tf.keras.utils.to_categorical(full_y["digit_2"].iloc[train_idx].values, num_classes=len(target_encoder.dic_cat["digit_2"].keys())),
        tf.keras.utils.to_categorical(full_y["digit_3"].iloc[train_idx].values, num_classes=len(target_encoder.dic_cat["digit_3"].keys()))
    )    
    train_ds = create_dataset(tmp_doc_embed, tmp_target, batch_size, True)

    with tf.device('/CPU:0'):
        tmp_doc_embed = pad_sequences(doc_vectorize(full_x["sentence"].to_numpy()[val_idx]).numpy(), padding='pre').astype(dtype="float32")
    tmp_target = (
        tf.keras.utils.to_categorical(full_y["digit_1"].iloc[val_idx].values, num_classes=len(target_encoder.dic_cat["digit_1"].keys())),
        tf.keras.utils.to_categorical(full_y["digit_2"].iloc[val_idx].values, num_classes=len(target_encoder.dic_cat["digit_2"].keys())),
        tf.keras.utils.to_categorical(full_y["digit_3"].iloc[val_idx].values, num_classes=len(target_encoder.dic_cat["digit_3"].keys()))
    )
    val_ds = create_dataset(tmp_doc_embed, tmp_target, batch_size, False)
    
    global loss_weight
    # loss weight tuning
    if len(loss_weight) == 0:
        optuna_study = create_study(direction='maximize', sampler=TPESampler())
        optuna_study.optimize(lambda trial: optuna_objective_function(trial, train_ds, val_ds), n_trials=100)
        loss_weight = (
            optuna_study.best_params["loss_weight1"],
            optuna_study.best_params["loss_weight2"],
            optuna_study.best_params["loss_weight3"]
        )

    cb_tensorboard = tf_callbacks.TensorBoard(log_dir="./tmp_tb", histogram_freq=1)
    try:
        wandb.tensorboard.patch(root_logdir="./tmp_tb")
    except:
        pass
    wandb.init(
        project="KOSTAT_IndustrialStandard_" + model_name,
        group=wandb_group_name,
        name="fold_" + str(fold)
    ); wandb.config.step = 0
    cb_earlyStopping = tf_callbacks.EarlyStopping(patience=patient_epochs, monitor='val_weighted_acc', mode='max')
    cb_reduceLR = tf_callbacks.ReduceLROnPlateau(patience=patient_lr, factor=0.5, min_lr=1e-5)
    cb_modelsave = tf_callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_weighted_acc', mode='max',
                                                save_weights_only=True, save_best_only=True)

    model = Model_Mutlitask(create_model(), loss_weight=loss_weight)
    model.compile(
        optimizer=tfa.optimizers.AdamW(learning_rate=eta, weight_decay=weight_decay)
    )

    print("start training")
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=0,
            callbacks=[cb_reduceLR, cb_earlyStopping, cb_modelsave, cb_tensorboard, TqdmCallback(verbose=0)])
    model.load_weights(checkpoint_filepath)
    print("end training")
    model_list.append(model)

    history_list.append(history)
    fold_metric.append(model.evaluate(val_ds, return_dict=True, verbose=0))
    print(fold_metric[-1])
    del train_ds, val_ds

    with tf.device('/CPU:0'):
        tmp_doc_embed = pad_sequences(doc_vectorize(test_x["sentence"].to_numpy()).numpy(), padding='pre').astype(dtype="float32")
    test_ds = create_dataset(tmp_doc_embed, None, batch_size, False)

    prob1, prob2, prob3 = model.model_digit_base.predict(test_ds)
    result_prob["digit_1"] += prob1 / n_folds
    result_prob["digit_2"] += prob2 / n_folds
    result_prob["digit_3"] += prob3 / n_folds
    
    wandb.log({"fold": fold,
               "weighted_logloss": fold_metric[-1]["weighted_logloss"],
               "weighted_acc": fold_metric[-1]["weighted_acc"],
               "weighted_f1score": fold_metric[-1]["weighted_f1score"],
               "loss_weight": loss_weight})

    wandb.finish()

    print("Fold " + str(fold) + " Time (minutes) : ", round((time() - tmp_time) / 60, 3))

In [49]:
model_list = []
history_list = []
fold_metric = []
seed_everything()

n_folds = 20
base_train_size = 1024
tr_size = base_train_size * 250
kfolds_spliter = StratifiedShuffleSplit(n_folds, train_size=tr_size, test_size=int(tr_size * 0.1), random_state=1234)
# n_folds = 2
# base_train_size = 1024
# tr_size = base_train_size * 2
# batch_size = 256
# kfolds_spliter = ShuffleSplit(n_folds, train_size=tr_size, test_size=int(tr_size * 0.25), random_state=1)
# n_folds = 10
# # base_train_size = 1024
# # tr_size = base_train_size * 1
# batch_size = 256
# kfolds_spliter = StratifiedKFold(n_folds, shuffle=True, random_state=1004)

start_time_training = time()
# fold training
for fold, (train_idx, val_idx) in enumerate(kfolds_spliter.split(range(full_y.shape[0]), full_y["digit_3"])):
    start_mem = memory_usage()
    do_fold_training(fold, train_idx, val_idx)
    tf.keras.backend.clear_session()
    gc.collect()
    end_mem = memory_usage()
    print("@Memory leaked :", end_mem - start_mem)
end_time_training = time()

[debug] memory usage:   3341.836 MB

===== Fold  0 =====

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/models/NN_tfidf_multi_task_learningdoc_vectorizer_vocab4096_0/assets


wandb: Currently logged in as: frony (use `wandb login --relogin` to force relogin)


start training


0epoch [00:00, ?epoch/s]

end training
{'digit1_logloss': 0.16217510402202606, 'digit1_acc': 0.9610156416893005, 'digit1_f1score': 0.8575485348701477, 'digit2_logloss': 0.33332473039627075, 'digit2_acc': 0.9203125238418579, 'digit2_f1score': 0.6609205007553101, 'digit3_logloss': 0.49425140023231506, 'digit3_acc': 0.8865625262260437, 'digit3_f1score': 0.5684502124786377, 'weighted_logloss': 0.4349726438522339, 'weighted_acc': 0.9001186490058899, 'weighted_f1score': 0.5900899767875671, 'loss': 0.8923866748809814, 'regularization_loss': 0.02129277214407921, 'total_loss': 0.9136794209480286}



...

start training


0epoch [00:00, ?epoch/s]

end training
{'digit1_logloss': 0.1624210923910141, 'digit1_acc': 0.9599218964576721, 'digit1_f1score': 0.8623411059379578, 'digit2_logloss': 0.3324383795261383, 'digit2_acc': 0.9201953411102295, 'digit2_f1score': 0.675835907459259, 'digit3_logloss': 0.4987724721431732, 'digit3_acc': 0.8832812309265137, 'digit3_f1score': 0.5760791301727295, 'weighted_logloss': 0.4454424977302551, 'weighted_acc': 0.8975623846054077, 'weighted_f1score': 0.606028139591217, 'loss': 0.9362894892692566, 'regularization_loss': 0.02411908656358719, 'total_loss': 0.9604085683822632}



fold,▁
global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/epoch_digit1_acc,▁▅▆▇▇██████████▇▇▇▇▇▆▆▅▅▄▃▃▂▂
train/epoch_digit1_f1score,▄▆▇▇█████████████▇▇▆▆▅▃▃▂▂▂▁▁
train/epoch_digit1_logloss,▇▃▃▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▅▅▆▇▇██
train/epoch_digit2_acc,▁▄▅▆▇▇██████████▇▇▇▇▆▆▅▄▄▃▃▂▂
train/epoch_digit2_f1score,▂▄▅▆▇▇████████▇▇▇▇▇▆▅▅▄▃▂▂▂▁▁
train/epoch_digit2_logloss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▅▆▆▇▇██
train/epoch_digit3_acc,▁▄▅▆▇▇██████████▇▇▇▇▆▆▅▅▄▃▃▃▂
train/epoch_digit3_f1score,▂▄▅▆▇▇██████████▇▇▇▆▆▅▄▃▃▂▂▁▁
train/epoch_digit3_logloss,█▅▄▃▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▅▅▆▆▇▇█


Fold 19 Time (minutes) :  9.875
[debug] memory usage:  16499.312 MB
@Memory leaked : 421.515625


In [50]:
# tf.keras.backend.clear_session()
# gc.collect()
# wandb.finish()

# validation score summary

In [51]:
wandb.init(
    project="KOSTAT_IndustrialStandard_" + model_name,
    name="summary"
); wandb.config.step = 0

In [52]:
trainable_count = model_size[0]
non_trainable_count = model_size[1]
training_total_time_minutes = end_time_training - start_time_training
wandb.log({"epochs": epochs, "patient_epochs": patient_epochs, "batch_size": batch_size, "patient_lr": patient_lr,
           "optimizer": optimizer_name, "eta": eta, "weight_decay": weight_decay, "dropoutRate": dropoutRate, "training_total_time_minutes": training_total_time_minutes,
           "model_img_path": wandb.Image(model_img_path), "total_params": trainable_count + non_trainable_count, "trainable_count": trainable_count, "non_trainable_count": non_trainable_count})

In [53]:
wandb.finish()

batch_size,▁
dropoutRate,▁
epochs,▁
eta,▁
non_trainable_count,▁
patient_epochs,▁
patient_lr,▁
total_params,▁
trainable_count,▁
training_total_time_minutes,▁
weight_decay,▁


# Inference

In [54]:
result_pred = dict.fromkeys(["digit_1", "digit_2", "digit_3"])
for i in result_prob.keys():
    dataframe(result_prob[i]).to_csv("/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/submission/" + model_name + "_" + "_prob.csv",
                          encoding="cp949", index=False)
    result_pred[i] = result_prob[i].argmax(axis=1)

In [55]:
for i in result_pred.keys():
    tmp = []
    for j in result_pred[i]:
        tmp.append(list(target_encoder.dic_cat[i].values()).index(j))
    result_pred[i] = array(list(target_encoder.dic_cat[i].keys()))[tmp]

In [56]:
submission = read_csv("/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/답안 작성용 파일.csv", encoding="cp949")

In [57]:
submission["digit_1"] = result_pred["digit_1"]
submission["digit_2"] = result_pred["digit_3"]
submission["digit_3"] = result_pred["digit_3"]

In [58]:
submission.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,561,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,466,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,949,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,952,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,562,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"


In [59]:
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/projects/other_competition/통계청_산업분류경진대회_NLP/submission/" + model_name + "_" + ".csv",
                  encoding="cp949", index=False)